# 第9章: ベクトル空間法 (I)

enwiki-20150112-400-r10-105752.txt.bz2は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる． メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパスenwiki-20150112-400-r100-10576.txt.bz2を用いよ．

In [50]:
import numpy as np

# 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

+ トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
+ 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [51]:
with open('enwiki-20150112-400-r100-10576.txt', 'r') as f:
    delete_words = ".,!?;:()[]'" + '"'
    corpus = []
    for line in f.readlines():
        for token in line[:-1].split():
            if token[0] in delete_words:
                token = token[1:]
            if len(token) == 0:
                break
            if token[-1] in delete_words:
                token = token[:-1]
            if token:
                corpus.append(token)
with open('corpus.txt', 'w') as f:
    f.write(" ".join(corpus))

# 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [52]:
# http://www.projectvisa.com/fullcountrylist.asp
with open('country.txt', 'r') as f:
    countries = [x[:-1] for x in f.readlines() if " " in x]

with open('corpus.txt', 'r') as f:
    corpus = f.read()[:-1]

In [53]:
for c in countries:
    corpus = corpus.replace(c, c.replace(" ", "_"))

In [54]:
corpus_list = corpus.split(" ")
dictionary = np.array(list(set(corpus_list)))
T = len(corpus_list)

# 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

+ ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
+ 単語$t$を選ぶ度に，文脈幅$d$は$\{1,2,3,4,5\}$の範囲でランダムに決める．

In [55]:
def get_d():
    d = np.random.randint(-5, 5)
    if d == 0:
        d = get_d()
    return d

with open("82.txt", "w") as f:
    for i, t in enumerate(corpus_list):
        d = get_d() 
        if (i + d >= 0) and  (i + d < T):
            c = corpus_list[i + d]
            if c != t:
                line = t + "\t" + c + "\n"
                f.write(line)

# 83. 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

+ $f(t,c)$: 単語$t$と文脈語$c$の共起回数
+ $f(t,∗)$: 単語$t$の出現回数
+ $f(∗,c)$: 文脈語$c$の出現回数
+ $N$: 単語と文脈語のペアの総出現回数

In [56]:
import pandas as pd
tc = pd.read_csv("82.txt", sep="\t", header=None, names=["t", "c"])
tc = tc[tc.t != tc.c]

In [57]:
N = len(tc.index)
N

11261966

In [58]:
tc.head()

,t,c
0,a,that
1,political,is
2,philosophy,societies
3,that,often
4,advocates,philosophy


In [59]:
f_t = tc.groupby('t').count()
f_c = tc.groupby('c').count()
tc['count'] = tc['t']
f_tc = tc.groupby(list("tc")).count()

In [60]:
f_t.ix['a']

c    221358
Name: a, dtype: int64

In [61]:
f_c.ix['a']

t    220661
Name: a, dtype: int64

In [62]:
f_tc.ix[('a', 'that')][0]

1690

# 84. 単語文脈行列の作成
83の出力を利用し，単語文脈行列$X$を作成せよ．ただし，行列$X$の各要素$X_{tc}$は次のように定義する．

+ $f(t,c)≥10$ならば，$X_{tc}=PPMI(t,c)=max\{log \frac{N×f(t,c)}{f(t,∗)×f(∗,c)},0\}$
+ $f(t,c)<10$ならば，$X_{tc}=0$

ここで，$PPMI(t,c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．
なお，行列$X$の行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列$X$のほとんどの要素は$0$になるので，非$0$の要素だけを書き出せばよい．

In [63]:
f_tc[f_tc['count'] >= 10].head()

count
t  c             
#1 the         15
$1 million     13
&  A           12
   Co          22
   Company     15

In [64]:
len(f_tc[f_tc['count'] >= 10])

98075

In [65]:
from scipy import sparse
from math import log
W = len(dictionary)
X = sparse.lil_matrix((W, W))

In [66]:
dictionary[100]

'Donglin'

In [67]:
np.where(dictionary == 'is')[0][0]

301318

In [68]:
dictionary[310116]

'ɗ'

In [69]:
X_df = f_tc[f_tc['count'] >= 10].reset_index()
X_df.head()

,t,c,count
0,#1,the,15
1,$1,million,13
2,&,A,12
3,&,Co,22
4,&,Company,15


In [70]:
f_t = f_t.reset_index()
f_t.columns = [['t', 'ft']]

In [71]:
f_c = f_c.reset_index()
f_c.columns = [['c', 'fc']]

In [72]:
X_df = pd.merge(X_df, f_t, on='t')
X_df = pd.merge(X_df, f_c, on='c')
X_df.head()

,t,c,count,ft,fc
0,#1,the,15,175,685907
1,&,the,141,3840,685907
2,),the,28,406,685907
3,-,the,193,4652,685907
4,.,the,23,565,685907


In [73]:
X_df['ppmi1'] = (N * X_df['count']) / (X_df['ft'] * X_df['fc'])

In [74]:
X_df.head()

,t,c,count,ft,fc,ppmi1
0,#1,the,15,175,685907,1.407350
1,&,the,141,3840,685907,0.602888
2,),the,28,406,685907,1.132351
3,-,the,193,4652,685907,0.681187
4,.,the,23,565,685907,0.668388


In [ ]:
len(X_df[X_df['ppmi1'] > 1])

69920

In [ ]:
with open('X.txt', 'w') as f:
    line = "t_i,c_i,ppmi\n"
    f.write(line)
    for row in X_df[X_df['ppmi1'] > 1].iterrows():
        t = row[1]['t']
        c = row[1]['c']
        t_i = np.where(dictionary == t)[0][0]
        c_i = np.where(dictionary == c)[0][0]
        ppmi = log(row[1]['ppmi1'])
        line = str(t_i) + "," + str(c_i) + "," + str(ppmi) + "\n"
        f.write(line)

In [ ]:
X_csv = pd.read_csv('X.txt')
for row in X_csv.iterrows():
    X[row[1]['t_i'], row[1]['c_i']] = row[1]['ppmi']

# 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

In [ ]:
# https://stackoverflow.com/questions/33603787/performing-pca-on-large-sparse-matrix-by-using-sklearn
from sklearn.decomposition import TruncatedSVD
clf = TruncatedSVD(300)
Xpca = clf.fit_transform(X)

In [ ]:
Xpca

# 86. 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [ ]:
def vec(word):
    i = np.where(dictionary == word)[0][0]
    return Xpca[i].reshape(1, -1)

In [ ]:
vec("United_States")

# 87. 単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_word_cs(word1, word2):
    return cosine_similarity(vec(word1), vec(word2))[0][0]
get_word_cs("United_States", "U.S")

# 88. 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [ ]:
def get_top10(word):
    lst=[]
    for i, w in enumerate(Xpca[:1000]):
        cs = cosine_similarity(vec(word), w.reshape(1, -1))[0][0]
        lst.append((dictionary[i], cs)
    return pd.DataFrame(lst)

In [ ]:
eng_top10 = get_top10("England")

In [ ]:
eng_top10.sort_values(1).head(10)

# 89. 加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ

In [ ]:
vec89 = vec("Spain") - vec("Madrid") + vec("Athens")
lst=[]
for i, w in enumerate(Xpca[:1000]):
    cs = cosine_similarity(vec89, w.reshape(1, -1))
    lst.append((dictionary[i], cs[0][0]))
df89 = pd.DataFrame(lst)

In [ ]:
df89.sort_values(1).head(10)